In [8]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [9]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),
                                  columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1])
                                                                                         if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

import pandas as pd


def get_human_dataset( ):
   
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv('./human_activity/features.txt',sep='\s+',
                        header=None,names=['column_index','column_name'])
   
    # 중복된 피처명을 수정하는 get_new_feature_name_df()를 이용, 신규 피처명 DataFrame생성.
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
   
    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
   
    # 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
    X_train = pd.read_csv('./human_activity/train/X_train.txt',sep='\s+', names=feature_name )
    X_test = pd.read_csv('./human_activity/test/X_test.txt',sep='\s+', names=feature_name)
   
    # 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여
    y_train = pd.read_csv('./human_activity/train/y_train.txt',sep='\s+',header=None,names=['action'])
    y_test = pd.read_csv('./human_activity/test/y_test.txt',sep='\s+',header=None,names=['action'])
   
    # 로드된 학습/테스트용 DataFrame을 모두 반환
    return X_train, X_test, y_train, y_test




X_train, X_test, y_train, y_test = get_human_dataset()

In [10]:
# RandomForest 일반화 성능 확인
from sklearn.ensemble import RandomForestClassifier

rf_clf=RandomForestClassifier(random_state=0, max_depth=8)
# 학습
rf_clf.fit(X_train, y_train)

# 테스트데이터(new data)에 대한 예측값 추출
pred=rf_clf.predict(X_test)

# 테스트데이터에 대한 예측값의 정확도 추출
acc=accuracy_score(pred, y_test) 

print(f'랜덤 포레스트의 정확도 : {acc:.4f}')

랜덤 포레스트의 정확도 : 0.9220


In [7]:
# GridSearchCV
from sklearn.model_selection import GridSearchCV
params={
    'n_estimators':[100,500]
    , 'max_depth':[6,8,10,12]
    , 'min_samples_leaf':[8,12,18]
    , 'min_samples_split':[8,16,20]
}

rf_clf=RandomForestClassifier(random_state=0,n_jobs=-1)
grid_cv=GridSearchCV(rf_clf, param_grid=params, cv=5, n_jobs=-1)

grid_cv.fit(X_train,y_train)

print(f'정확도 : {grid_cv.best_score_}')
print(f'최적 하이퍼 파라미터 : \n', grid_cv.best_params_)

정확도 : 0.9272358569532504
최적 하이퍼 파라미터 : 
 {'max_depth': 12, 'min_samples_leaf': 12, 'min_samples_split': 8, 'n_estimators': 500}


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf1=RandomForestClassifier(
    n_estimators=100
    , min_samples_leaf=6
    , max_depth=16
    , min_samples_split=2
    , random_state=0
)
rf_clf1.fit(X_train, y_train)

pred=rf_clf1.predict(X_test)
accuracy_score(y_test, pred)

0.9253478113335596

: 